In [1]:
import pandas as pd
# Configurar para exibir todas as colunas
pd.set_option('display.max_columns', None)

import numpy as np

import sqlite3

from unidecode import unidecode

import warnings
warnings.filterwarnings("ignore")

In [2]:
# Passo 1: Conectando ao arquivo .db
caminho = r'data\raw\futebol.db'
conexao = sqlite3.connect(caminho)

# Passo 2: Criando um objeto cursor
cursor = conexao.cursor()

# Passo 3: Executando a consulta SQL para listar as tabelas
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")

# Passo 4: Recuperando os resultados da consulta
tabelas = cursor.fetchall()

# Passo 5: Imprimindo os nomes das tabelas
for tabela in tabelas:
    print(tabela[0])

# Fechando a conexão com o banco de dados
#conexao.close()

fixture
players_statistics
leagues


In [3]:
# Consulta SQL para selecionar todos as ligas
consulta_sql = "SELECT * FROM leagues;"

# Lendo os dados da tabela e salvando em um DataFrame do pandas
df_leagues = pd.read_sql_query(consulta_sql, conexao)

#display(df_leagues[df_leagues['countryname'] != 'World'].drop_duplicates(['league_id']).head())

paises = ['England', 
          'Spain', 
          'France', 
          #'Brazil', 
          'Germany', 
          'Italy', 
          'Netherlands', 
          'Argentina']

{'England' : [39], 
 'Spain' : [140],
 'France': [61],
 'Brazil' : [71], 
 'Germany' : [78],
 'Italy' : [135],
 'Netherlands' : [88],
 'Argentina' : [128]}

#df_leagues[df_leagues['countryname'].isin(paises[6:7])].drop_duplicates(subset=['league_name'])

{'England': [39],
 'Spain': [140],
 'France': [61],
 'Brazil': [71],
 'Germany': [78],
 'Italy': [135],
 'Netherlands': [88],
 'Argentina': [128]}

In [4]:
paises

['England', 'Spain', 'France', 'Germany', 'Italy', 'Netherlands', 'Argentina']

In [5]:
liga_num = 39

In [6]:
filtro = df_leagues['league_id'] == liga_num
liga = df_leagues[filtro].head(1)

nome_liga = str(liga_num) + "_" + liga['countryname'].item() + "_" + unidecode(liga['league_name'].item().replace(" ", "_"))

nome_liga


'39_England_Premier_League'

In [7]:
# Consulta SQL para selecionar todos os valores da tabela 'fixture' para a Serie A
consulta_sql = f"SELECT * \
                FROM fixture \
                WHERE league_id = {liga_num};"

#consulta_sql = f"SELECT * \
#                FROM fixture;"

# Lendo os dados da tabela e salvando em um DataFrame do pandas
df_fixture = pd.read_sql_query(consulta_sql, conexao)

In [300]:
# Consulta SQL para selecionar todos os valores da tabela 'players_statistics' para a Serie A
consulta_sql =  f"SELECT * \
                FROM players_statistics \
                WHERE fixture_id IN (SELECT DISTINCT fixture_id FROM fixture WHERE league_id = {liga_num});"

# Lendo os dados da tabela e salvando em um DataFrame do pandas
df_players_statistics = pd.read_sql_query(consulta_sql, conexao)

# Conferindo valores duplicados

df_players_statistics['fixture_id'] = df_players_statistics['fixture_id'].astype(int)
df_players_statistics['player_id'] = df_players_statistics['player_id'].astype(int)

print(f'Existem ids de jogos duplicados: {any(df_fixture.fixture_id.duplicated())}')
print('')

df = df_players_statistics[df_players_statistics.duplicated(subset=['fixture_id', 'player_id'], keep=False)]
#display(df)

new_id = df_players_statistics['player_id'].max()

for row in df['player_id'].unique():

    # pegando quantidade de nomes associado ao id 
    nomes = df['player_name'][df['player_id'] == row].unique().tolist()
    nomes

    for nome in nomes:
        new_id += 1
        df_players_statistics['player_id'][(df_players_statistics['player_id'] == row) & (df_players_statistics['player_name'] == nome)] = new_id

df = df_players_statistics[df_players_statistics.duplicated(subset=['fixture_id', 'player_id'], keep=False)]
#display(df)

new_id = df_players_statistics['player_id'].max()

#fixture_id = 150609
#player_id = 417958
for _, row in df.iterrows():
    fixture_id = row['fixture_id']
    player_id = row['player_id']

    indexs = df_players_statistics[(df_players_statistics['fixture_id'] == fixture_id) & (df_players_statistics['player_id'] == player_id)].index[:-1]

    for index in indexs:
        new_id += 1
        df_players_statistics.loc[index, 'player_id'] = new_id

df = df_players_statistics[df_players_statistics.duplicated(subset=['fixture_id', 'player_id'], keep=False)]
df


Existem ids de jogos duplicados: False



,fixture_id,team_id,team_name,team_logo,team_update,player_id,player_name,player_photo,statistics_offsides,statistics_games_minutes,statistics_games_number,statistics_games_position,statistics_games_rating,statistics_games_captain,statistics_games_substitute,statistics_shots_total,statistics_shots_on,statistics_goals_total,statistics_goals_conceded,statistics_goals_assists,statistics_goals_saves,statistics_passes_total,statistics_passes_key,statistics_passes_accuracy,statistics_tackles_total,statistics_tackles_blocks,statistics_tackles_interceptions,statistics_duels_total,statistics_duels_won,statistics_dribbles_attempts,statistics_dribbles_success,statistics_dribbles_past,statistics_fouls_drawn,statistics_fouls_committed,statistics_cards_yellow,statistics_cards_red,statistics_penalty_won,statistics_penalty_commited,statistics_penalty_scored,statistics_penalty_missed,statistics_penalty_saved


In [301]:
# criando mapa de id's
map_player_id = df_players_statistics[['player_id']].apply(pd.Series.unique) 
map_player_id = pd.DataFrame(data={
    'player_api_id': map_player_id.player_id,
    'player_node_id': pd.RangeIndex(len(map_player_id)),
})

map_player_id = map_player_id.astype({'player_api_id' : 'int',
                                      'player_node_id' : 'int'})

map_player_id = pd.concat([map_player_id, pd.DataFrame({'player_api_id' : [map_player_id['player_api_id'].max() + 1, map_player_id['player_api_id'].max() + 2],                                                        
                                       'player_node_id': [map_player_id['player_node_id'].max() + 1, map_player_id['player_node_id'].max() + 2]
                                       })], ignore_index=True)

print("Mapa de player ID:")
print("==========================================")
print(map_player_id.head())
print()

map_team_id = np.unique(np.concatenate([df_fixture['teams_home_id'], df_fixture['teams_away_id']]))
map_team_id = pd.DataFrame(data={
    'team_api_id': map_team_id,
    'team_node_id': pd.RangeIndex(len(map_team_id)),
})

map_team_id = map_team_id.astype({'team_api_id' : 'int',
                                  'team_node_id' : 'int'})

print("Mapa de team ID:")
print("==========================================")
print(map_team_id.head())
print()

Mapa de player ID:
   player_api_id  player_node_id
0            882               0
1            887               1
2            885               2
3            889               3
4            891               4

Mapa de team ID:
   team_api_id  team_node_id
0           33             0
1           34             1
2           35             2
3           36             3
4           37             4



In [302]:
df_teams = pd.DataFrame({'teams_name' : pd.concat([df_fixture['teams_home_name'], df_fixture['teams_away_name']], ignore_index=True).to_list(),
'teams_id' : pd.concat([df_fixture['teams_home_id'], df_fixture['teams_away_id']], ignore_index=True).to_list()}).drop_duplicates(subset=['teams_id'])

df_teams = df_teams.astype({'teams_id' : 'int'})


In [303]:
df = df_fixture[['teams_home_name', 'teams_away_name', 'league_country', 
                  'league_name', 'fixture_date', 'goals_home', 'goals_away', 
                  'league_season', 'league_round', 'fixture_id','teams_home_id', 'teams_away_id']]

df = df.dropna(subset=['goals_home'])
df = df.dropna(subset=['goals_away'])

df = df.astype({'fixture_id' : 'int',
                'teams_home_id' : 'int',
                'teams_away_id' : 'int',
                'goals_home' : 'int',
                'goals_away' : 'int'})


df_players_statistics = df_players_statistics.astype({'fixture_id' : "int",
                                                      'team_id' : "int",
                                                      'player_id' : 'int'})


colunas = {'teams_home_name': 'Home Team',
           'teams_away_name': 'Away Team',
           'league_country': 'Country',
           'league_name': 'League',
           'fixture_date' : 'Kick Off',
           'goals_home' : 'Home Score',
           'goals_away': 'Away Score'}

df.rename(columns=colunas, inplace=True)

df["Kick Off"] = pd.to_datetime(df['Kick Off'])
df["ts"] = df["Kick Off"].apply(lambda x: x.timestamp() * 1000)
df['Kick Off'] = df['Kick Off'].dt.strftime('%Y-%m-%d %H:%M:%S %Z')

df = df.sort_values(by='ts', ascending=True)
df = df.reset_index(drop = True)

df = df.dropna(subset=['Home Score'])

df['league_round'] = df['league_round'].str.split("-").str[-1]
df['league_round'] = pd.to_numeric(df['league_round'], errors='coerce')
df['league_round'].fillna(df['league_round'].max() + 1, inplace=True)

df = df.astype({'Home Score' : 'float',
                'Away Score': 'float',
                'league_season': 'float',
                'league_round': 'float'})

df['Home Team'] = np.where(df['Home Team'] == 'Vasco DA Gama', 'Vasco Da Gama', df['Home Team'])
df['Away Team'] = np.where(df['Away Team'] == 'Vasco DA Gama', 'Vasco Da Gama', df['Away Team'])

df['teams_home_id'] = df['teams_home_id'].map(map_team_id.set_index('team_api_id')['team_node_id'])
df['teams_away_id'] = df['teams_away_id'].map(map_team_id.set_index('team_api_id')['team_node_id'])

df = df[df['fixture_id'].isin(df_players_statistics['fixture_id'].unique())].sort_values(by='ts')

df_players_statistics['player_id'] = df_players_statistics['player_id'].map(map_player_id.set_index('player_api_id')['player_node_id'])
df_players_statistics['team_id'] = df_players_statistics['team_id'].map(map_team_id.set_index('team_api_id')['team_node_id'])

original_columns = df.columns

for i in ['home', 'away']:
    for j in range(11):
        df[f'player_{i}_{j}'] = None

df_raw = df.copy()

df.sort_values(by='ts').head()

,Home Team,Away Team,Country,League,Kick Off,Home Score,Away Score,league_season,league_round,fixture_id,teams_home_id,teams_away_id,ts,player_home_0,player_home_1,player_home_2,player_home_3,player_home_4,player_home_5,player_home_6,player_home_7,player_home_8,player_home_9,player_home_10,player_away_0,player_away_1,player_away_2,player_away_3,player_away_4,player_away_5,player_away_6,player_away_7,player_away_8,player_away_9,player_away_10
227,Chelsea,Manchester City,England,Premier League,2015-01-31 17:30:00 UTC,1.0,1.0,2014.0,23.0,192904,16,17,1.422725e+12,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
228,Arsenal,Aston Villa,England,Premier League,2015-02-01 13:30:00 UTC,5.0,0.0,2014.0,23.0,192905,9,26,1.422797e+12,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
288,Swansea,Liverpool,England,Premier League,2015-03-16 20:00:00 UTC,0.0,1.0,2014.0,29.0,192966,32,7,1.426536e+12,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
380,Manchester United,Tottenham,England,Premier League,2015-08-08 11:45:00 UTC,1.0,0.0,2015.0,1.0,192297,0,14,1.439034e+12,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
381,Everton,Watford,England,Premier League,2015-08-08 14:00:00 UTC,2.0,2.0,2015.0,1.0,192301,12,5,1.439042e+12,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None


In [304]:
colunas_casa = df.filter(like='player_home_').columns.to_list()
colunas_fora = df.filter(like='player_away_').columns.to_list()

jogos = []

for indice, linha in df.iterrows():
    home = linha['teams_home_id']
    away = linha['teams_away_id']
    match = linha['fixture_id']
    
    players_home = df_players_statistics['player_id'][(df_players_statistics['fixture_id'] == match) & (df_players_statistics['team_id'] == home)].iloc[:11].values
    players_away = df_players_statistics['player_id'][(df_players_statistics['fixture_id'] == match) & (df_players_statistics['team_id'] == away)].iloc[:11].values

    if (len(players_home) != 11):
        jogos.append(match)
        players_home = np.concatenate((players_home, [map_player_id['player_node_id'].max()] * (11 - len(players_home))))
        
    if (len(players_away) != 11):
        jogos.append(match)
        players_away = np.concatenate((players_away, [map_player_id['player_node_id'].max()] * (11 - len(players_away))))
        

    indices_linhas = df.index[df['fixture_id'] == match]

    df.loc[indices_linhas, colunas_casa] = players_home
    df.loc[indices_linhas, colunas_fora] = players_away

df = df[~df.isnull().any(axis=1)]


In [305]:
df

,Home Team,Away Team,Country,League,Kick Off,Home Score,Away Score,league_season,league_round,fixture_id,teams_home_id,teams_away_id,ts,player_home_0,player_home_1,player_home_2,player_home_3,player_home_4,player_home_5,player_home_6,player_home_7,player_home_8,player_home_9,player_home_10,player_away_0,player_away_1,player_away_2,player_away_3,player_away_4,player_away_5,player_away_6,player_away_7,player_away_8,player_away_9,player_away_10
227,Chelsea,Manchester City,England,Premier League,2015-01-31 17:30:00 UTC,1.0,1.0,2014.0,23.0,192904,16,17,1.422725e+12,1388,796,327,1389,126,1390,346,135,1391,138,1392,237,1394,307,1395,1396,1397,271,1398,308,200,275
228,Arsenal,Aston Villa,England,Premier League,2015-02-01 13:30:00 UTC,5.0,0.0,2014.0,23.0,192905,9,26,1.422797e+12,1402,252,1403,436,305,259,1404,188,1405,258,304,1409,1410,1411,31,1412,437,369,300,1413,107,1414
288,Swansea,Liverpool,England,Premier League,2015-03-16 20:00:00 UTC,0.0,1.0,2014.0,29.0,192966,32,7,1.426536e+12,209,1417,1418,1419,553,341,244,34,189,1420,1421,1425,1426,1427,101,323,207,1428,406,274,208,1099
380,Manchester United,Tottenham,England,Premier League,2015-08-08 11:45:00 UTC,1.0,0.0,2015.0,1.0,192297,0,14,1.439034e+12,1431,1,343,1432,4,1433,186,8,1434,318,1435,363,266,293,45,46,48,654,53,51,1437,52
381,Everton,Watford,England,Premier League,2015-08-08 14:00:00 UTC,2.0,2.0,2015.0,1.0,192301,12,5,1.439042e+12,1477,182,267,183,1478,495,1479,1480,132,437,13,1484,1485,399,141,143,1486,145,1487,1488,1489,149
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3305,Aston Villa,Bournemouth,England,Premier League,2023-03-18 15:00:00 UTC,3.0,0.0,2022.0,28.0,868217,26,2,1.679152e+12,669,724,615,420,1346,556,561,1006,514,727,726,1238,57,224,1239,677,1351,1315,352,1181,119,470
3306,Brentford,Leicester,England,Premier League,2023-03-18 15:00:00 UTC,1.0,1.0,2022.0,28.0,868218,20,13,1.679152e+12,941,1195,944,240,949,1236,947,1018,950,951,957,1194,21,15,1356,745,20,974,1358,22,454,976
3307,Southampton,Tottenham,England,Premier League,2023-03-18 15:00:00 UTC,3.0,3.0,2022.0,28.0,868224,8,14,1.679152e+12,1217,430,376,1219,972,231,287,1045,234,188,542,500,995,48,1214,1372,429,335,46,190,55,52
3308,Chelsea,Everton,England,Premier League,2023-03-18 17:30:00 UTC,2.0,2.0,2022.0,28.0,868220,16,12,1.679161e+12,125,793,1222,1279,616,1365,303,18,752,1278,573,181,182,184,239,510,306,144,187,1235,350,23


In [306]:
caminho = r''

pd.merge(map_player_id,
         df_players_statistics[['player_name', 'player_id']].drop_duplicates(),
         left_on=['player_node_id'],
         right_on=['player_id'],
         how='left').drop(columns=['player_id']).to_excel(caminho + "\map_player_id_" + nome_liga + ".xlsx", index=False)

pd.merge(map_team_id,
         df_teams,
         left_on=['team_api_id'],
         right_on=['teams_id'],
         how='left').drop(columns=['teams_id']).to_excel(caminho + "\map_team_id_" + nome_liga + ".xlsx", index=False)

df.to_excel(caminho + "\matchs_" + nome_liga + ".xlsx", index=False)
